Datasets help in collecting data from production, staging, evaluations and manually

In [1]:
import phoenix as px
px.launch_app()

/media/uberdev/ddrv/telemetenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/uberdev/.phoenix


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


running experiment evaluations |██████████| 2/2 (100.0%) | ⏳ 00:35<00:00 | 17.78s/it


In [2]:
import pandas as pd
from dotenv import load_dotenv
load_dotenv("/media/uberdev/ddrv/gitFolders/python_de_learners_data/.env")

True

In [3]:
# simple data point

sdf = pd.DataFrame(
    [{
        "question": "What is paul graham famous for",
        "answer": "Co-founding Y-combinator and writing on startups",
        "metadata": ["topic", "tech"]
    }]
)

In [4]:
phoenix_client = px.Client()

In [6]:
dataset = phoenix_client.upload_dataset(
    dataframe=sdf,
    dataset_name="sample-dataset",
    input_keys=["question"],
    output_keys=["answer"],
    metadata_keys=["metadata"]
)

📤 Uploading dataset...
💾 Examples uploaded: http://localhost:6006/datasets/RGF0YXNldDox/examples
🗄️ Dataset version ID: RGF0YXNldFZlcnNpb246MQ==


In [7]:
from openai import OpenAI
from phoenix.experiments.types import Example
# above import gets the datatypes under Experiments

In [9]:
import os
openai_client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [10]:
task_prompt = "Answer in few words: {question}"

In [11]:
def task(example: Example) -> str:
    question = example.input["question"]
    messages_context = task_prompt.format(question=question)
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini", 
        messages= [
                {
                    "role": "user",
                    "content": messages_context
                }
        ]
    )
    return response.choices[0].message.content

In [12]:
# we can look at couple of evaluators built in to phoenix
from phoenix.experiments.evaluators import (
    ContainsAllKeywords,
    ConcisenessEvaluator,
)
# pull the model classes that are build for evaluation
from phoenix.evals.models import OpenAIModel
contains_kw = ContainsAllKeywords(keywords=["Y Combinator", "YC"])
model = OpenAIModel(model="gpt-4o-mini")
conciseness = ConcisenessEvaluator(model=model)

In [16]:
# define custom similarity with Code
from typing import Any, Dict

def jaccard_similarity(output: str, expected: Dict[str, Any]) -> float:
    actual_words = set(output.lower().split(" "))
    expected_words = set(expected["answer"].lower().split(" "))
    words_in_common = actual_words.intersection(expected_words)
    all_words = actual_words.union(expected_words)
    # https://en.wikipedia.org/wiki/Jaccard_index
    return len(words_in_common) / len(all_words)
    

In [14]:
from phoenix.experiments.evaluators import create_evaluator
from typing import Any, Dict

eval_prompt_template = """
Given the QUESTION and REFERENCE_ANSWER, determine whether the ANSWER is accurate.
Output only a single word (accurate or inaccurate).

QUESTION: {question}

REFERENCE_ANSWER: {reference_answer}

ANSWER: {answer}

ACCURACY (accurate / inaccurate):
"""


@create_evaluator(kind="llm")  # need the decorator or the kind will default to "code"
def accuracy(input: Dict[str, Any], 
             output: str, 
             expected: Dict[str, Any]) -> float:
    # message contents are manually built
    message_content = eval_prompt_template.format(
        question=input["question"],
        reference_answer=expected["answer"], 
        answer=output
    )
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", 
                   "content": message_content}]
    )
    response_message_content = response.choices[0].message.content.lower().strip()
    return 1.0 if response_message_content == "accurate" else 0.0

In [17]:
# run experiment, which requires dataset, task, evaluators
from phoenix.experiments import run_experiment
experiment = run_experiment(
    dataset=dataset, # phoenix object
    task=task,
    experiment_name="sample-expt",
    evaluators=[jaccard_similarity, accuracy],
)

WARNI [phoenix.evals.executors] 🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


🧪 Experiment started.
📺 View dataset experiments: http://localhost:6006/datasets/RGF0YXNldDox/experiments
🔗 View this experiment: http://localhost:6006/datasets/RGF0YXNldDox/compare?experimentId=RXhwZXJpbWVudDoy


running tasks |██████████| 1/1 (100.0%) | ⏳ 00:01<00:00 |  1.43s/it
WARNI [phoenix.evals.executors] 🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


✅ Task runs completed.
🧠 Evaluation started.


running experiment evaluations |██████████| 2/2 (100.0%) | ⏳ 00:00<00:00 |  2.22it/s


🔗 View this experiment: http://localhost:6006/datasets/RGF0YXNldDox/compare?experimentId=RXhwZXJpbWVudDoy

Experiment Summary (08/16/24 10:12 AM +0530)
--------------------------------------------
            evaluator  n  n_scores  avg_score
0            accuracy  1         1       1.00
1  jaccard_similarity  1         1       0.25

Tasks Summary (08/16/24 10:12 AM +0530)
---------------------------------------
   n_examples  n_runs  n_errors
0           1       1         0


In [18]:
px.close_app()